In [3]:
# Use GCN model instead of MLP
# Use same framework as DemoVAE6FederatedMLP but don't add any noise

ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [59]:
sites = ['Dallas', 'Detroit', 'Chicago', 'Baltimore']
test_site = 'Hartford'

train_fcs = [[], [], [], []]
train_labels = [[], [], [], []]

test_fcs = []
test_labels = []

import pickle
import numpy as np

diag_map = {'SZP': 0, 'BPP': 1, 'SADBPP': 2, 'SADDEPP': 2, 'NC': 3, 'SADDEPR': 3, 'SADBPR': 3, 'BPR': 3, 'SZR': 3} 

bsnipdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/BSNIP/'
bsnip = pickle.load(open(f'{bsnipdir}/demographics.pkl', 'rb'))

a,b = np.triu_indices(264,1)

for sub in bsnip['Age_cal']:
    parts = sub.split('.')
    diag = bsnip['DXGROUP_1'][sub]
    # if diag != 'SZP' and diag != 'NC':
    #     continue
    diag = diag_map[diag]
    if diag != 0 and diag != 3:
        continue
    diag = int(diag == 3)
    p = np.load(f'{bsnipdir}/fc/{sub}_task-unk_fc.npy')
    p = remap(vec2mat(p))[a,b]
    if parts[0] == test_site:
        test_fcs.append(p)
        test_labels.append(diag)
    else:
        for i in range(len(sites)):
            if parts[0] == sites[i]:
                train_fcs[i].append(p)
                train_labels[i].append(diag)
                break

for i in range(len(sites)):
    train_fcs[i] = np.stack(train_fcs[i])
    train_labels[i] = np.array(train_labels[i])
    print(train_fcs[i].shape)
    print(train_labels[i])

test_fcs = np.stack(test_fcs)
test_labels = np.array(test_labels)
print(test_fcs.shape)
print(test_labels)

(162, 34716)
[1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0
 1 1 1 1 1 1 0 1 1 1 1 0 1 1]
(84, 34716)
[1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1
 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 1 1 1
 1 0 0 0 1 1 1 1 0 1]
(161, 34716)
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1
 1 0 0 1 1 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1
 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 1 0]
(226, 34716)
[0 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0

In [60]:
import torch

xtr_np = np.concatenate(train_fcs)
xtr = torch.from_numpy(xtr_np).float().cuda()
A = torch.zeros(len(xtr),len(xtr)).float().cuda()

for i in range(len(xtr)):
    if i % 50 == 0:
        print(i)
    for j in range(len(xtr)):
        if i == j:
            A[i,j] = 200
            continue
        s = np.dot(xtr_np[i], xtr_np[j])/np.linalg.norm(xtr_np[i])/np.linalg.norm(xtr_np[j])
        A[i,j] = s
    A[i,:] = A[i,:]/torch.sum(A[i,:])

print(A)


0
50
100
150
200
250
300
350
400
450
500
550
600
tensor([[0.3631, 0.0012, 0.0011,  ..., 0.0011, 0.0011, 0.0011],
        [0.0010, 0.3130, 0.0012,  ..., 0.0013, 0.0012, 0.0012],
        [0.0010, 0.0013, 0.3403,  ..., 0.0012, 0.0011, 0.0012],
        ...,
        [0.0010, 0.0014, 0.0012,  ..., 0.3214, 0.0012, 0.0013],
        [0.0010, 0.0013, 0.0011,  ..., 0.0013, 0.3359, 0.0012],
        [0.0010, 0.0013, 0.0011,  ..., 0.0013, 0.0011, 0.3248]],
       device='cuda:0')


In [61]:
import torch
import torch.nn as nn
import torch.nn.functional as F

ytr = torch.from_numpy(np.concatenate(train_labels)).long().cuda()

in_channels = 34716
out_channels = 2

class GIN(nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.fc1 = nn.Linear(in_channels, 100).float().cuda()
        self.fc2 = nn.Linear(100, out_channels).float().cuda()

    def forward(self, x, A):
        x = F.relu(self.fc1(x))
        x = A@x
        x = self.fc2(x)
        return x

model = GIN()
optim = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
ce = torch.nn.CrossEntropyLoss()

nepochs = 5000
pperiod = 200

for e in range(nepochs):
    optim.zero_grad()
    yhat = model(xtr, A)
    loss = ce(yhat, ytr)
    loss.backward()
    optim.step()
    if e % pperiod == 0 or e == nepochs-1:
        print(f'{e} {loss}')

print('Done')

0 0.6953985095024109
200 0.42394137382507324
400 0.415350079536438
600 0.41131913661956787
800 0.40715911984443665
1000 0.40397560596466064
1200 0.400769978761673
1400 0.397787481546402
1600 0.39502179622650146
1800 0.39200758934020996
2000 0.38935884833335876
2200 0.38646364212036133
2400 0.38370028138160706
2600 0.06759282946586609
2800 0.022321704775094986
3000 0.011056612245738506
3200 0.004707163665443659
3400 0.0031998902559280396
3600 0.0025270225014537573
3800 0.0021203707437962294
4000 0.0018388828029856086
4200 0.0016287666512653232
4400 0.0014654103433713317
4600 0.0013343013124540448
4800 0.0012271783780306578
4999 0.0011381165822967887
Done


In [62]:
xt = torch.from_numpy(test_fcs).float().cuda()
xt = torch.cat([xt, xtr])
xt_np = xt.detach().cpu().numpy()

At = torch.zeros(len(xt),len(xt)).float().cuda()

for i in range(len(xt)):
    if i % 50 == 0:
        print(i)
    for j in range(len(xt)):
        if i == j:
            At[i,j] = 100
            continue
        s = np.dot(xt_np[i], xt_np[j])/np.linalg.norm(xt_np[i])/np.linalg.norm(xt_np[j])
        At[i,j] = float(s)
    At[i,:] = At[i,:]/torch.sum(At[i,:])

yt = torch.from_numpy(test_labels).long().cuda()

print('Done')

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
Done


In [63]:
with torch.no_grad():
    yhat = model(xt, At)
    yhat = torch.argmax(yhat, dim=1)
    yhat = yhat[:len(yt)]
    acc = torch.sum(yhat == yt)/len(yt)
    print(acc)

tensor(0.8321, device='cuda:0')
